In [16]:
#All the imports & pyplot inline setting
import numpy as np
import pandas as pd
import random
from scipy.spatial import distance
from sklearn import preprocessing

In [23]:
n_file='project3_dataset2.txt'
#n_file='project3_dataset1.txt'
n_neighbours=5
n_folds = 10

In [24]:
df = pd.read_csv(n_file,header=None,delim_whitespace=True)
shuffled_idx = np.random.permutation(df.index)
df = df.reindex(shuffled_idx)
data_truth = df.iloc[:,-1].values

In [25]:
def find_label():
    global data_distance
    global test_truth
    global data_label
    idx = np.argpartition(data_distance, n_neighbours)[:,:n_neighbours]
    for i in range(idx.shape[0]):
        count0 = 0
        distance0 = 0
        count1 = 0
        distance1 = 0
        for k in range(idx.shape[1]):
            if(train_truth[idx[i][k]]==0):
                count0 += 1
                distance0 += 1/data_distance[i][idx[i][k]]
            elif(train_truth[idx[i][k]]==1):
                count1 += 1
                distance1 += 1/data_distance[i][idx[i][k]]
        if(count0 > count1):
            data_label[i] = 0
        elif(count1 > count0):
            data_label[i] = 1
        elif(distance0 > distance1):
            data_label[i] = 0
        else:
            data_label[i] = 1

In [26]:
def calculate_distance(test,train):
    global data_distance
    for i in range(df.shape[1]-1):
        if(df[i].dtype=='object'):
            le = preprocessing.LabelEncoder()
            le.fit(df[:][i])
            l_test = le.transform(test[:][i])
            l_test = l_test.reshape((l_test.shape[0], 1))
            l_train = le.transform(train[:][i])
            l_train = l_train.reshape((l_train.shape[0], 1))
            dm = distance.cdist(l_test, l_train, lambda u, v: 0 if u==v else 1)
        else:
            l_test = test[:][i]
            l_test = l_test.reshape((l_test.shape[0], 1))
            l_train = train[:][i]
            l_train = l_train.reshape((l_train.shape[0], 1))
            dm = distance.cdist(l_test, l_train, 'euclidean')
        dm = (dm-np.amin(dm))/(np.amax(dm)-np.amin(dm))
        data_distance = np.add(data_distance,dm)

In [27]:
def calculate_measure(data_label,test_truth):
    total = data_label.shape[0]
    match = data_label[data_label.flatten()==test_truth.flatten()]
    non_match = data_label[data_label.flatten()!=test_truth.flatten()]
    pos = match[match==1].shape[0]
    neg = match[match==0].shape[0]
    false_pos = non_match[non_match==1].shape[0]
    false_neg = non_match[non_match==0].shape[0]
    cal_accuracy = (0.0 + pos + neg)/(0.0 + pos + neg + false_pos + false_neg)
    cal_precision = (0.0 + pos)/(0.0 + pos + neg)
    cal_recall = (0.0 + pos)/(0.0 + pos + false_neg)
    denom = (0.0 + cal_precision + cal_recall)
    if denom==0:
        denom=1
    cal_f_measure = (2.0*cal_precision*cal_recall)/denom
    return (cal_accuracy,cal_precision,cal_recall,cal_f_measure)

In [28]:
accuracy = 0.0
precision = 0.0
recall = 0.0
f_measure = 0.0
count=shuffled_idx.shape[0]//n_folds
for i in range(n_folds):
    test = df.iloc[count*i:count*(i+1)]
    train = pd.concat([df.iloc[0:count*i],df.iloc[count*(i+1):]])
    test_truth = data_truth[count*i:count*(i+1)]
    train_truth = pd.concat([df.iloc[:,-1].iloc[0:count*i],df.iloc[:,-1].iloc[count*(i+1):]]).values
    data_distance = np.zeros((test.shape[0],train.shape[0]))
    data_label = np.zeros(test_truth.shape)
    calculate_distance(test,train)
    find_label()
    current_accuracy,current_precision,current_recall,current_f_measure = calculate_measure(data_label,test_truth)
    print 'kFold ' + str(i) + ':\nAccuracy: ' + str(current_accuracy) + '\tPrecision: ' + str(current_precision) + \
    '\tRecall: ' + str(current_recall) + '\tF-measure: ' + str(current_f_measure)
    accuracy = accuracy + current_accuracy
    precision = precision + current_precision
    recall = recall + current_recall
    f_measure = f_measure + current_f_measure
print 'Average Accuracy: ' + str(accuracy/n_folds)
print 'Average Precision: ' + str(precision/n_folds)
print 'Average Recall: ' + str(recall/n_folds)
print 'Average F-Measure: ' + str(f_measure/n_folds)

kFold 0:
Accuracy: 0.608695652174	Precision: 0.178571428571	Recall: 0.277777777778	F-measure: 0.217391304348
kFold 1:
Accuracy: 0.586956521739	Precision: 0.333333333333	Recall: 0.529411764706	F-measure: 0.409090909091
kFold 2:
Accuracy: 0.804347826087	Precision: 0.0810810810811	Recall: 0.333333333333	F-measure: 0.130434782609
kFold 3:
Accuracy: 0.673913043478	Precision: 0.161290322581	Recall: 0.333333333333	F-measure: 0.217391304348
kFold 4:
Accuracy: 0.782608695652	Precision: 0.194444444444	Recall: 0.4375	F-measure: 0.269230769231
kFold 5:
Accuracy: 0.673913043478	Precision: 0.225806451613	Recall: 0.4375	F-measure: 0.297872340426
kFold 6:
Accuracy: 0.673913043478	Precision: 0.258064516129	Recall: 0.444444444444	F-measure: 0.326530612245
kFold 7:
Accuracy: 0.586956521739	Precision: 0.296296296296	Recall: 0.380952380952	F-measure: 0.333333333333
kFold 8:
Accuracy: 0.652173913043	Precision: 0.133333333333	Recall: 0.266666666667	F-measure: 0.177777777778
kFold 9:
Accuracy: 0.695652173913	